In [22]:
import numpy as np
import pickle
np.random.seed(7)  # for reproducibility


In [23]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2


In [24]:
import keras
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('tf')

In [25]:
def unpickle(file):
    with open(file,'rb') as fo:
        dict=pickle.load(fo, encoding='bytes')
    data=dict['data']
    data = data.reshape((-1, 32, 32, 3))
    labels=dict['label']
    return data, labels

In [26]:
def saveTest(your_predict_labels):
    f=open('testlabel.pickle','rb')
    labels=pickle.load(f)
    f.close()
  #here you must replace the labels that you have predicted for the test set:
    for i in range(len(labels)):
        labels[i]=your_predict_labels[i]
  #save the pickle file that you should upload:
    f=open('testlabel.pickle','wb')
    pickle.dump(labels,f)
    f.close()

In [27]:
X_train_label, Y_train_label =unpickle("trainset.pickle")
print(X_train.shape)

(9000, 32, 32, 3)


In [28]:
X_validation_label, Y_validation_label=unpickle("validset.pickle")

In [81]:
f=open('testset.pickle','rb')
dict=pickle.load(f)
f.close()
X_test_label=dict['data']
X_test_label = X_test_label.reshape((-1, 32, 32, 3))
X_test = X_test_label.astype('float32')

print(X_test_label.shape)

(3000, 32, 32, 3)


In [78]:
f=open('testlabel.pickle','rb')
Y_test_label=pickle.load(f)
f.close()
Y_test=np_utils.to_categorical(Y_test_label, 10)


In [31]:
X_train = X_train_label.astype('float32')
X_validation = X_validation_label.astype('float32')


Y_train = np_utils.to_categorical(Y_train_label, 10)
Y_validation = np_utils.to_categorical(Y_validation_label, 10)
print(X_train.shape)

(9000, 32, 32, 3)


In [86]:
# Parameter
num_classes = Y_train.shape[1]
batch_size = 64
epochs = 100
no_estimators = 5

In [87]:
# create CNN
def create_cnn_model():
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, input_shape=(32, 32, 3), activation='relu', border_mode='same'))
    model.add(BatchNormalization())    
    model.add(Dropout(0.2))
    model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Flatten())
    
    model.add(Dropout(0.2))
    model.add(Dense(1024, activation='relu', W_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu', W_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

In [89]:
## ENSEMBLE MODELS
# Train many CNN models
for i in np.arange(no_estimators):
    # check point to save best model
    checkpoint = keras.callbacks.ModelCheckpoint('models/model_'+str(i)+'.h5', monitor='val_acc', \
            verbose=0, save_best_only=True, save_weights_only=True, mode='auto')
    # clear GPU memory
    model = None
    K.clear_session()
    
    # train model
    model = create_cnn_model()
    model.fit(X_train, Y_train, validation_data=(X_validation, Y_validation), nb_epoch=100, \
                batch_size=64, verbose=2, callbacks=[checkpoint])
    pred=model.evaluate(X_test, Y_test)
    print(pred[1])

C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., activation="relu", padding="same")`
  after removing the cwd from sys.path.
C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  import sys
C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  del sys.path[0]
C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packa

Train on 9000 samples, validate on 3000 samples
Epoch 1/100
 - 8s - loss: 14.1620 - acc: 0.4509 - val_loss: 6.6040 - val_acc: 0.4983
Epoch 2/100
 - 4s - loss: 3.5552 - acc: 0.5540 - val_loss: 2.0162 - val_acc: 0.5263
Epoch 3/100
 - 4s - loss: 1.5576 - acc: 0.5876 - val_loss: 1.3823 - val_acc: 0.5813
Epoch 4/100
 - 4s - loss: 1.2216 - acc: 0.6094 - val_loss: 1.2486 - val_acc: 0.5600
Epoch 5/100
 - 4s - loss: 1.1246 - acc: 0.6401 - val_loss: 1.0848 - val_acc: 0.6253
Epoch 6/100
 - 4s - loss: 1.0997 - acc: 0.6472 - val_loss: 1.1164 - val_acc: 0.6017
Epoch 7/100
 - 4s - loss: 1.0880 - acc: 0.6633 - val_loss: 1.1282 - val_acc: 0.6043
Epoch 8/100
 - 4s - loss: 1.0440 - acc: 0.6824 - val_loss: 1.0326 - val_acc: 0.6663
Epoch 9/100
 - 4s - loss: 1.0133 - acc: 0.6953 - val_loss: 1.1214 - val_acc: 0.6207
Epoch 10/100
 - 4s - loss: 0.9885 - acc: 0.6960 - val_loss: 1.0351 - val_acc: 0.6607
Epoch 11/100
 - 4s - loss: 0.9713 - acc: 0.7123 - val_loss: 1.1364 - val_acc: 0.6437
Epoch 12/100
 - 4s - loss

Epoch 97/100
 - 4s - loss: 0.1699 - acc: 0.9721 - val_loss: 1.3666 - val_acc: 0.7033
Epoch 98/100
 - 4s - loss: 0.1585 - acc: 0.9734 - val_loss: 1.4745 - val_acc: 0.6857
Epoch 99/100
 - 4s - loss: 0.1640 - acc: 0.9733 - val_loss: 1.3545 - val_acc: 0.6970
Epoch 100/100
 - 4s - loss: 0.1579 - acc: 0.9737 - val_loss: 1.4840 - val_acc: 0.6863
3000/3000 [==============================] - 1s 177us/step
0.30333333325386047
Train on 9000 samples, validate on 3000 samples
Epoch 1/100
 - 8s - loss: 14.1065 - acc: 0.4594 - val_loss: 6.2092 - val_acc: 0.4960
Epoch 2/100
 - 4s - loss: 3.5314 - acc: 0.5503 - val_loss: 2.1514 - val_acc: 0.5023
Epoch 3/100
 - 4s - loss: 1.5653 - acc: 0.5830 - val_loss: 1.3820 - val_acc: 0.5947
Epoch 4/100
 - 4s - loss: 1.2384 - acc: 0.6007 - val_loss: 1.7346 - val_acc: 0.4463
Epoch 5/100
 - 4s - loss: 1.1108 - acc: 0.6268 - val_loss: 1.1652 - val_acc: 0.5830
Epoch 6/100
 - 4s - loss: 1.0560 - acc: 0.6421 - val_loss: 1.1067 - val_acc: 0.5943
Epoch 7/100
 - 4s - loss: 1

Epoch 92/100
 - 4s - loss: 0.1687 - acc: 0.9728 - val_loss: 1.2866 - val_acc: 0.7103
Epoch 93/100
 - 4s - loss: 0.1740 - acc: 0.9708 - val_loss: 1.4223 - val_acc: 0.7043
Epoch 94/100
 - 4s - loss: 0.1616 - acc: 0.9731 - val_loss: 1.2267 - val_acc: 0.7203
Epoch 95/100
 - 4s - loss: 0.1713 - acc: 0.9708 - val_loss: 1.3033 - val_acc: 0.7133
Epoch 96/100
 - 4s - loss: 0.1685 - acc: 0.9736 - val_loss: 1.3924 - val_acc: 0.7040
Epoch 97/100
 - 4s - loss: 0.1581 - acc: 0.9729 - val_loss: 1.2261 - val_acc: 0.7157
Epoch 98/100
 - 4s - loss: 0.1579 - acc: 0.9749 - val_loss: 1.4176 - val_acc: 0.6963
Epoch 99/100
 - 4s - loss: 0.1590 - acc: 0.9728 - val_loss: 1.4710 - val_acc: 0.6903
Epoch 100/100
 - 4s - loss: 0.1552 - acc: 0.9749 - val_loss: 1.3469 - val_acc: 0.7087
3000/3000 [==============================] - 1s 177us/step
0.4013333334128062
Train on 9000 samples, validate on 3000 samples
Epoch 1/100
 - 8s - loss: 14.0003 - acc: 0.4470 - val_loss: 6.0010 - val_acc: 0.5177
Epoch 2/100
 - 4s - los

Epoch 87/100
 - 4s - loss: 0.1939 - acc: 0.9659 - val_loss: 1.2212 - val_acc: 0.7027
Epoch 88/100
 - 4s - loss: 0.1680 - acc: 0.9727 - val_loss: 1.3306 - val_acc: 0.7017
Epoch 89/100
 - 4s - loss: 0.1579 - acc: 0.9744 - val_loss: 1.3295 - val_acc: 0.6960
Epoch 90/100
 - 4s - loss: 0.1761 - acc: 0.9698 - val_loss: 1.3566 - val_acc: 0.7013
Epoch 91/100
 - 4s - loss: 0.1716 - acc: 0.9729 - val_loss: 1.3955 - val_acc: 0.6900
Epoch 92/100
 - 4s - loss: 0.1633 - acc: 0.9721 - val_loss: 1.4038 - val_acc: 0.7007
Epoch 93/100
 - 4s - loss: 0.1770 - acc: 0.9697 - val_loss: 1.3922 - val_acc: 0.6893
Epoch 94/100
 - 4s - loss: 0.1539 - acc: 0.9742 - val_loss: 1.2838 - val_acc: 0.7000
Epoch 95/100
 - 4s - loss: 0.1557 - acc: 0.9723 - val_loss: 1.3283 - val_acc: 0.7097
Epoch 96/100
 - 4s - loss: 0.1710 - acc: 0.9708 - val_loss: 1.3147 - val_acc: 0.6940
Epoch 97/100
 - 4s - loss: 0.1626 - acc: 0.9716 - val_loss: 1.3475 - val_acc: 0.6997
Epoch 98/100
 - 4s - loss: 0.1666 - acc: 0.9706 - val_loss: 1.361

Epoch 82/100
 - 4s - loss: 0.1971 - acc: 0.9696 - val_loss: 1.4356 - val_acc: 0.6873
Epoch 83/100
 - 4s - loss: 0.2018 - acc: 0.9664 - val_loss: 1.4116 - val_acc: 0.7027
Epoch 84/100
 - 4s - loss: 0.1815 - acc: 0.9716 - val_loss: 1.3436 - val_acc: 0.6910
Epoch 85/100
 - 4s - loss: 0.1782 - acc: 0.9691 - val_loss: 1.2296 - val_acc: 0.7073
Epoch 86/100
 - 4s - loss: 0.1721 - acc: 0.9722 - val_loss: 1.3221 - val_acc: 0.6923
Epoch 87/100
 - 4s - loss: 0.1671 - acc: 0.9724 - val_loss: 1.3089 - val_acc: 0.7020
Epoch 88/100
 - 4s - loss: 0.1647 - acc: 0.9742 - val_loss: 1.4099 - val_acc: 0.6880
Epoch 89/100
 - 4s - loss: 0.1770 - acc: 0.9710 - val_loss: 1.3431 - val_acc: 0.7000
Epoch 90/100
 - 4s - loss: 0.1740 - acc: 0.9686 - val_loss: 1.2270 - val_acc: 0.6963
Epoch 91/100
 - 4s - loss: 0.1728 - acc: 0.9730 - val_loss: 1.3605 - val_acc: 0.6997
Epoch 92/100
 - 4s - loss: 0.1499 - acc: 0.9766 - val_loss: 1.2869 - val_acc: 0.6987
Epoch 93/100
 - 4s - loss: 0.1592 - acc: 0.9739 - val_loss: 1.328

Epoch 77/100
 - 4s - loss: 0.2306 - acc: 0.9663 - val_loss: 1.3499 - val_acc: 0.6823
Epoch 78/100
 - 4s - loss: 0.2167 - acc: 0.9696 - val_loss: 1.2556 - val_acc: 0.7090
Epoch 79/100
 - 4s - loss: 0.2146 - acc: 0.9668 - val_loss: 1.3257 - val_acc: 0.6960
Epoch 80/100
 - 4s - loss: 0.2084 - acc: 0.9674 - val_loss: 1.3718 - val_acc: 0.6907
Epoch 81/100
 - 4s - loss: 0.2028 - acc: 0.9708 - val_loss: 1.3089 - val_acc: 0.7003
Epoch 82/100
 - 4s - loss: 0.1853 - acc: 0.9734 - val_loss: 1.3001 - val_acc: 0.6953
Epoch 83/100
 - 4s - loss: 0.2083 - acc: 0.9691 - val_loss: 1.2668 - val_acc: 0.6917
Epoch 84/100
 - 4s - loss: 0.2029 - acc: 0.9689 - val_loss: 1.3451 - val_acc: 0.7043
Epoch 85/100
 - 4s - loss: 0.1888 - acc: 0.9729 - val_loss: 1.4217 - val_acc: 0.6860
Epoch 86/100
 - 4s - loss: 0.1803 - acc: 0.9737 - val_loss: 1.3546 - val_acc: 0.6970
Epoch 87/100
 - 4s - loss: 0.1922 - acc: 0.9667 - val_loss: 1.2304 - val_acc: 0.6967
Epoch 88/100
 - 4s - loss: 0.1685 - acc: 0.9726 - val_loss: 1.331

In [82]:
# Check individual scores - val_acc and get list of the best models
score_list = []
score_list2 = []
score_list3=[]
model = create_cnn_model()
for i in np.arange(no_estimators):    
    model.load_weights('models/model_'+str(i)+'.h5')
    scores3= model.evaluate(X_test, Y_test, verbose=0)
    scores2= model.evaluate(X_train, Y_train, verbose=0)
    scores = model.evaluate(X_validation, Y_validation, verbose=0)
    score_list.append(scores[1])   #validation Acc
    score_list2.append(scores2[1])    #Train Acc
    score_list3.append(scores3[1])  #Test Acc
score_list = np.array(score_list)
score_list2= np.array(score_list2)
score_list3= np.array(score_list3)
score_list_filter = score_list >= 0.72 # threshold to select best model
print("Train Acc: "+str(score_list2))
print("Validation Acc: "+ str(score_list))
print("Test Acc: "+ str(score_list3))

print(score_list_filter)

C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., activation="relu", padding="same")`
  after removing the cwd from sys.path.
C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  import sys
C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  del sys.path[0]
C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packa

Train Acc: [0.99877778 0.99866667 0.99766667 0.99888889 0.994      0.99744444
 0.99966667 0.98955556 0.999      0.999     ]
Validation Acc: [0.71466667 0.71766667 0.70566667 0.70933333 0.72333333 0.70333333
 0.70966667 0.724      0.70866667 0.72233333]
Test Acc: [0.40166667 0.437      0.426      0.40733333 0.36433333 0.47933333
 0.39233333 0.41433333 0.407      0.406     ]
[False False False False  True False False  True False  True]


In [85]:
# Ensemble predictions ~ val_err 16%
predictions = np.zeros((Y_validation.shape[0],Y_validation.shape[1]))
predictions_test=np.zeros((Y_test.shape[0],Y_test.shape[1]))
model = create_cnn_model()
for i in np.arange(no_estimators): 
    if score_list_filter[i] == False:
        continue
    #validation
    model.load_weights('models/model_'+str(i)+'.h5')
    prediction = model.predict_classes(X_validation, batch_size=200, verbose=0)
    prediction_onehot = np_utils.to_categorical(prediction, 10)
    predictions = predictions + prediction_onehot*score_list[i]
    #Test
    prediction_test = model.predict_classes(X_test, batch_size=200, verbose=0)
    prediction_onehot_test = np_utils.to_categorical(prediction_test, 10)
    predictions_test = predictions_test + prediction_onehot_test*score_list3[i]

predictions_label = np.argmax(predictions, axis=1)
evaluate = np.equal(predictions_label, Y_validation_label)

predictions_label_test = np.argmax(predictions_test, axis=1)
evaluate_test = np.equal(predictions_label_test, Y_test_label)

print("Ensemble Error(validation): %.2f%%" % (100-np.mean(evaluate)*100))
print("Ensemble Accuracy(validation): %.2f%%" % (np.mean(evaluate)*100))

print("Ensemble Error(Test): %.2f%%" % (100-np.mean(evaluate_test)*100))
print("Ensemble Accuracy(Test): %.2f%%" % (np.mean(evaluate_test)*100))

C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., activation="relu", padding="same")`
  after removing the cwd from sys.path.
C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  import sys
C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  del sys.path[0]
C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packa

Ensemble Error(validation): 24.50%
Ensemble Accuracy(validation): 75.50%
Ensemble Error(Test): 60.50%
Ensemble Accuracy(Test): 39.50%


In [83]:
## PERFORM PREDICTION ON TEST SET

X_test = X_test_label.astype('float32')

predictions = np.zeros((Y_validation.shape[0],Y_validation.shape[1]))
model = create_cnn_model()
for i in np.arange(no_estimators): 
    if score_list_filter[i] == False: # skip bad model
        continue
    model.load_weights('models/model_'+str(i)+'.h5')
    prediction = model.predict_classes(X_test, batch_size=200, verbose=0)
    prediction_onehot = np_utils.to_categorical(prediction, 10)
    predictions = predictions + prediction_onehot*score_list[i]

predictions_label = np.argmax(predictions, axis=1)
output = predictions_label.tolist()

with open('testlabel.pickle','wb') as f:
    pickle.dump(output, f)

pred = model.evaluate(X_test, Y_test)
print("test loss: ",pred[0]+"test acc: ",pred[1])

C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., activation="relu", padding="same")`
  after removing the cwd from sys.path.
C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  import sys
C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  del sys.path[0]
C:\Users\PEM\Anaconda3\envs\bms-tf(gpu)\lib\site-packa

3000/3000 [==============================] - 2s 666us/step


TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')